In [1]:
cd ~/Desktop/DePaul/CSC528ComputerVision/FinalProject

/Users/taihuali/Desktop/DePaul/CSC528ComputerVision/FinalProject


In [2]:
import numpy as np 
import pandas as pd 
import os, dicom, fnmatch
from scipy.misc import imsave
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
allslices = pd.read_csv('parsedXMLnew.csv')

In [4]:
def getDicom(directory):
    matches = []
    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*.dcm'):
            matches.append(os.path.join(root, filename))
    return matches

In [5]:
dcm_images = getDicom('LIDC-IDRI/')
dcm_images[0]

'LIDC-IDRI/LIDC-IDRI-0001/1.3.6.1.4.1.14519.5.2.1.6279.6001.298806137288633453246975630178/1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636438705059720603192/000001.dcm'

In [7]:
with_nodule = 0
no_nodule = 0
labeled = []

for dcm in dcm_images:
    temp = [dcm]
    dicom_file = dicom.read_file(dcm)
    image = dicom_file.pixel_array
    # plt.imshow(image, cmap='gray')
    # plt.show()
    # find the slice that contains nodule
    if dicom_file.SOPInstanceUID in allslices['imageSOP_UID'].values:
        with_nodule+=1
        coords = allslices[allslices['imageSOP_UID'] == dicom_file.SOPInstanceUID]['coords']
        if coords.shape[0] == 1:
            coord = [x.split(';') for x in coords.values[0].split('|') if len(x)>2]
            coord = np.array(coord).astype('int')
        else:
            coord = [x.split(';') for x in coords.values[0].split('|') if len(x)>2]
            coord = np.array(coord).astype('int')
        left = coord.T[0].min() # left
        right = coord.T[0].max() # right
        top = coord.T[1].min() # top
        bottom = coord.T[1].max() # bottom
        nodule_patch = image[top:bottom, left:right]
        imsave(dcm.replace('.dcm', '.jpg'), nodule_patch)
        temp.append(1)
    else:
        no_nodule += 1
        temp.append(0)
    labeled.append(temp)

In [8]:
labeled = pd.DataFrame(labeled, columns=['Image', 'Label'])
labeled.head()

,Image,Label
0,LIDC-IDRI/LIDC-IDRI-0001/1.3.6.1.4.1.14519.5.2...,0
1,LIDC-IDRI/LIDC-IDRI-0001/1.3.6.1.4.1.14519.5.2...,0
2,LIDC-IDRI/LIDC-IDRI-0001/1.3.6.1.4.1.14519.5.2...,1
3,LIDC-IDRI/LIDC-IDRI-0001/1.3.6.1.4.1.14519.5.2...,0
4,LIDC-IDRI/LIDC-IDRI-0001/1.3.6.1.4.1.14519.5.2...,0


In [9]:
writer = pd.ExcelWriter('LIDC_Label.xlsx')
labeled.to_excel(writer, 'sheet1', index=False)
writer.save()

In [10]:
print 'in total: {} images'.format(len(dcm_images))
print '{} images have nodules and {} do not'.format(with_nodule, no_nodule)

in total: 1852 images
137 images have nodules and 1715 do not
